In [1]:
import numpy as np
from pathlib import Path
import cv2
import pandas as pd
from skimage import measure, filters, draw
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import tqdm 
from scipy import ndimage

import SimpleITK as sitk
import itkwidgets 
import napari
import copy

from napari.utils.colormaps import colormap_utils as cu

c:\Users\zoeba\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
def find_slice_with_mask(mask):
    '''
    Détecte sur les index des premières et dernière tranches d'un masque. 
    Args:
        mask (np array): le masque ou le fond est des 1 et le masque des 0
    Return:
        (rangemin,rangemax) (int,int): tuple avec l'index des premières et dernière tranches d'un masque. 
    '''
    nb_slices_with_masks=[]
    for nb_slice in range(mask.shape[0]-1, 0,-1):
        if 1 in mask[nb_slice]:
            nb_slices_with_masks.append(nb_slice)
    rangemin = np.min(nb_slices_with_masks) #mask 1
    rangemax = np.max(nb_slices_with_masks) #mask 1
    return (rangemin,rangemax)

In [52]:
def fit_ellipses(mask, rangemin, rangemax):
    ''' 
    Le but est de récuperer les paramètres d'une liste ellipse en fonction d'une liste de masques fournis par le network construction
    Args :
        mask (numpy array) : masque des artères
        rangemin (int): début des coupes sélectionnées
        rangemax (int): fin des coupes sélectionnées
    Return:
        list_params_ellipses_to_return (list[list[double]]): paramètres de chaque ellipse
    '''
    list_params_ellipses=[]
    
    for index_img in tqdm.tqdm(reversed(range(rangemax-rangemin))):
        img=mask[rangemin+index_img]
        contours_found,_=cv2.findContours(img.astype(np.uint8), method=cv2.RETR_LIST, mode=cv2.CHAIN_APPROX_SIMPLE)
        if len(contours_found)>0:
            ellipse=cv2.fitEllipse(contours_found[0])
            ellipse_ok=((ellipse[0][0], ellipse[0][1]), (ellipse[1][0]/2,ellipse[1][1]/2), ellipse[2])
            list_params_ellipses.append(ellipse_ok)
        else:
            print("erreur nb contours")
    list_params_ellipses.reverse()
    return list_params_ellipses

In [93]:
def predict_and_compare_values(list_params):
    #working on it
    new_list_params=[list_params[-1]]
    list_products = [my_tuple[1][0]*my_tuple[1][1] for my_tuple in list_params]
    list_indexes= [ i for i in range(len(list_products))]
    fit = np.polyfit(list_indexes, list_products, deg=1)
    predict_size_area = np.poly1d(fit)
    list_products_predicted=[predict_size_area(index) for index in list_indexes]
    
    count=0
    for index in reversed(range(len(list_products_predicted)-1)):
        product_predicted=list_products_predicted[index]
        if np.abs((product_predicted-list_products[index])*100/product_predicted)<50: #si moins de 10 pourcent d'erreurs, je garde ma value originales
            new_list_params.append(list_params[index])
        else: #sinon, je mets les param de l'ellipse précédente
            new_list_params.append(new_list_params[-1])
            count+=1
    print("count ",count)
    new_list_params.reverse()
    return new_list_params

In [120]:
def dilation(img_3D, kernel_size):
    ''' 
    Dilatation d'une image 
    Args:
        img_3D (numpy array): image à dilater
        kernel_size (int): taille du kernel
    '''
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_size,kernel_size))
    for index, slice in enumerate(img_3D):
        img_3D[index] = cv2.dilate(img_3D[index] , kernel, iterations=1)

In [123]:
def create_3d_ellipses(list_params_ellipses, shape_image, rangemin, rangemax, dilate=False):
    ''' 
    Draw 3d ellipses on a white image
    Args:
        list_params_ellipses (list[list[double]]): the parameters of each ellipse
        mask (numpy array): the initial masks
        rangemin (int): first slice of the mask
        rangemask (int): last slice of the mask
        dilate (Boolean): indicatinf if neede to dliate of not the image
    Return:
        ellipses_3d (numpy array): images with drawing of ellipses on it
    '''
    ellipses_3d = []
    for index_img in tqdm.tqdm(range(rangemax-rangemin)):
        ellipse_mask = np.zeros((shape_image[1],shape_image[2]), dtype=np.uint8)
        params = list_params_ellipses[index_img]
        center = (int(params[0][0]),int(params[0][1]))
        axis = (int(params[1][0]),int(params[1][1]))
        angle = int(params[2])
        color = 1
        cv2.ellipse(ellipse_mask, center, axis, angle, 0, 360, color,-1)
        ellipses_3d.append(ellipse_mask)
    numpy_ellipse_3d=np.array(ellipses_3d)
    if dilate:
        dilation(numpy_ellipse_3d, kernel_size=4)
    return numpy_ellipse_3d

In [124]:
def keeping_arteries_only(filled_ellipses, image):
    '''
    Le but est de garder l'image originale en la restreignant à l'intérieur des ellipses.
    Args:
        filled_ellipses (np.array): les ellipses remplies
        image (np.array): image d'origine 
    Return:
        list_arteries_only (np.array): l'image originale restreinte à l'intérieur des ellipses
    ''' 
    list_arteries_only=[]
    for index in range(len(filled_ellipses)):
        slice_ell=filled_ellipses[index]
        slice_im=image[index]
        arteries_only=np.where(slice_ell==1,slice_im, np.nan)
        list_arteries_only.append(arteries_only)
    return np.array(list_arteries_only)

In [125]:
def construct_ellipses_network(mask_list,range_to_do):
    #working on it 
    '''
    En partant de la liste de masques construite par la network construction, des ellipses sont approximées pour reconstruire un réseau artériel lisse. 
    Args:
        mask_list (list[np.array]): la liste des masque construite par la network construction
    Return:
        info_all_masks (list[dic]): une liste où pour chaque loigne (qui correspond à un masque), il y a un dictionnaire où les clefs sont: rangemin, rangemax, mask,  smooth_ellipses
        ellipses_network (np.array): l'image 3D où le réseau d'ellipse est reconstitué
    ''' 
    info_all_masks = []
    ellipses_network=np.zeros(mask_list[0].shape)
    for index_mask in (range(range_to_do)):#len(mask_list))):
        print(index_mask+1, " out of ", range_to_do)
        mask = mask_list[index_mask]
        mask_01 = np.where(mask == 255, 1, 0)
        rangemin,rangemax=find_slice_with_mask(mask_01)

        list_params=fit_ellipses(mask_01, rangemin, rangemax)
        
        ellipses_3d=create_3d_ellipses(list_params,mask.shape, rangemin, rangemax)
        
        
        new_list_param=predict_and_compare_values(list_params)
        new_ellipses_3d=create_3d_ellipses(new_list_param,mask.shape, rangemin, rangemax, dilate=True)
        return new_ellipses_3d, ellipses_3d

    #     before=np.zeros((rangemin, mask_01.shape[1],mask_01.shape[1]))
    #     after=np.zeros((mask_01.shape[0]-rangemax, mask_01.shape[1],mask_01.shape[1]))
    #     whole=np.concatenate((before,smooth_ellipses,after))
        
    #     ellipses_network=ellipses_network+whole

    #     ellipses_network_ok=np.where(ellipses_network>1,1,ellipses_network)

    #     info_of_one_mask={'index':index_mask, 'rangemin':rangemin,'rangemax':rangemax,'mask':mask_01, 'smooth_ellipses':smooth_ellipses}
    #     info_all_masks.append(info_of_one_mask)
    # return info_all_masks, ellipses_network_ok

In [8]:
def construct_mask_network(mask_list,range_to_do):
    '''
 
    ''' 
    mask_network=np.zeros(mask_list[0].shape)
    for index_mask in tqdm.tqdm(range(range_to_do)):#len(mask_list))):
        mask_ok=np.where(mask_list[index_mask]==255,0,1)
        
        mask_network=mask_network+mask_ok
        mask_network_ok=np.where(mask_network>1,1,mask_network)
    return mask_network_ok

In [9]:
def plotting_hist(images):
    #will be used
    '''
    En partant  des images 3D, on plot l'histogramme de chaque slice.
    Args:
        images (np.array)
    ''' 
    count=0
    for index in range(len(images)):
        slice_im=images[index]
        if 255 in slice_im and count <10:
            print(index,end=', ')
            # plt.hist(slice_im.flatten(), bins=255)
            # plt.title(index)
            # plt.ylim((1,20))
            # plt.xlim((1,256))
            # plt.show()
            # count+=1

# constructing the whole network

In [10]:
mask_list = np.load(Path("temp/mask_list.npy"))
range_to_do=1#len(mask_list)


In [121]:
new_ellipses3D,ellipses3D= construct_ellipses_network(mask_list,range_to_do) #
dilation(new_ellipses3D, 4)

1  out of  1


265it [00:00, 1049.45it/s]
100%|██████████| 265/265 [00:00<00:00, 6030.94it/s]


count  10


100%|██████████| 265/265 [00:00<00:00, 5126.75it/s]


In [122]:
viewer = napari.Viewer()
mask = mask_list[0]
mask_01 = np.where(mask == 255, 1, 0)
rangemin,rangemax=find_slice_with_mask(mask_01)

image_array = np.load(Path("temp/preprocessed_image.npy"))


image_layer = viewer.add_image( image_array[rangemin:rangemax], colormap="gray", contrast_limits=(0,255))
# image_layer = viewer.add_image(mask_01[rangemin:rangemax], colormap='green',blending="additive", contrast_limits=(0,1))

image_layer = viewer.add_image(new_ellipses3D, colormap="blue",blending="additive", contrast_limits=(0,1))
napari.run()

average testing

In [42]:
# network_arteries_only_no255=np.where(network_arteries_only==255, np.nan, network_arteries_only)
# network_mean_slice=[]
# print(len(network_arteries_only_no255))
# for index in range(len(network_arteries_only_no255)):
#     slice=network_arteries_only_no255[index]
#     network_mean_slice.append(np.nanmean(slice))

# moyenne_network = np.nanmean(network_mean_slice)
# std_network = np.nanstd(network_mean_slice)
# print("mean gray for all network: ", moyenne_network)
# print("std gray for all network: ",std_network)

# list_deviating_indices=[]
# for index in range(len(network_mean_slice)):
#     if network_mean_slice[index] < moyenne_network - 1.5*std_network:
#         list_deviating_indices.append(index)
# print(list_deviating_indices)